In [2]:
import sys
from os import path

import cv2
import numpy as np

from PyQt5 import QtCore
from PyQt5 import QtWidgets
from PyQt5 import QtGui

import pytesseract
from PIL import Image
from pytesseract import image_to_string
from gtts import gTTS
import os


In [3]:

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

tessdata_config = r'--tessdata-dir "C:\Program Files\Tesseract-OCR\tessdata"'

In [4]:
class RecordVideo(QtCore.QObject):
    image_data = QtCore.pyqtSignal(np.ndarray)

    def __init__(self, camera_port=0, parent=None):
        super().__init__(parent)
        self.camera = cv2.VideoCapture(camera_port)
        
        self.timer = QtCore.QBasicTimer()

    def start_recoding(self):
        self.timer.start(0, self)

    def timerEvent(self, event):
        if (event.timerId() != self.timer.timerId()):
            return
        read, data = self.camera.read()
        if read:
            self.image_data.emit(data)
    
    def saveframe(self):
        read, data = self.camera.read()
        if read:
            cv2.imwrite("a.png", data)
            image = Image.fromarray(data)
            image.load()

            text = pytesseract.image_to_string(image, lang='eng', config=tessdata_config)
            print('Text_Found:', text, len(text))

            if len(text)>0:
                text_to_speech = gTTS(text=text, lang='en')
                text_to_speech.save('converteVoice.mp3')
                os.system('start converteVoice.mp3')

In [5]:
# class for detection window
class DetectionWidget(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.image = QtGui.QImage()
        self._red = (0, 0, 255)
        self._width = 2
        self._min_size = (30, 30)

    def image_data_slot(self, image_data):
        self.image = self.get_qimage(image_data)
        if self.image.size() != self.size():
            self.setFixedSize(self.image.size())
        self.update()

    def get_qimage(self, image: np.ndarray):
        height, width, color = image.shape
        bytesPerLine = 3 * width
        QImage = QtGui.QImage

        image = QImage(
            image.data,
            width,
            height,
            bytesPerLine,
            QImage.Format_RGB888
        )

        image = image.rgbSwapped()
        return image
    
    def paintEvent(self, event):
        painter = QtGui.QPainter(self)
        painter.drawImage(0, 0, self.image)
        self.image = QtGui.QImage()

In [6]:
# FinalClass
class MainWidget(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.detection_widget = DetectionWidget()
        self.record_video = RecordVideo()

        image_slot = self.detection_widget.image_data_slot
        self.record_video.image_data.connect(image_slot)

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.detection_widget)
        self.run_button = QtWidgets.QPushButton('Start')
        layout.addWidget(self.run_button)
        
        self.run_button.clicked.connect(self.record_video.start_recoding)

        self.screenshot = QtWidgets.QPushButton('snap shot')
        layout.addWidget(self.screenshot)
        self.screenshot.clicked.connect(self.record_video.saveframe)
        self.setLayout(layout)

In [7]:
def run():
    app = QtWidgets.QApplication(sys.argv)
    main_window = QtWidgets.QMainWindow()
    main_widget = MainWidget()
    main_window.setCentralWidget(main_widget)
    main_window.show()

    sys.exit(app.exec_())

In [12]:
run()